# Преобразование данных через матричное умножение

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

1. [ЗАГРУЗКА ДАННЫХ](#shag_1)

2. [УМНОЖЕНИЕ МАТРИЦ](#shag_2)

3. [АЛГОРИТМ ПРЕОБРАЗОВАНИЯ](#shag_3)

4. [ПРОВЕРКА АЛГОРИТМА](#shag_4)


<a id='shag_1'></a>

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
display(df)
print(df.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


Видим, что нет пропущенных значений, типы значений - int и float. 

<a id='shag_2'></a>

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении на обратимую матрицу результат не изменится.

**Обоснование:** Предсказания:

$$
a = Xw
$$

Предсказания после умножения на обратимую матрицу (вместо матрицы Х теперь ХР):

$$
a = XРw
$$

Формула обучения:

$$
w2 = (XР^T XР)^{-1} XР^T y
$$

Отсюда:

$$
a2 = XР (XР^T XР)^{-1} XР^T y2
$$

Преобразования:

$$
XР (XР^T XР)^{-1} XР^T y = ХР ХР^{-1} ХР^T^{-1} XP^T y2 = 1 * ХР^T^{-1} XP^T * y2 = y2
$$

$$
X*P *((X*P)^T *(X*P))^{-1} (X*P)^T y = (Х*P) *(Х*P)^{-1} *((Х*P)^T)^{-1} (X*P)^T y2 = 1 * ((Х*P)^T)^{-1} (X*P)^T * y2
$$

$$
X*P *((X*P)^T *(X*P))^{-1} (X*P)^T y = (Х*P) *(Х*P)^{-1} *((Х*P)^T)^{-1} (X*P)^T y2 = 1 * ((Х*P)^T)^{-1} (X*P)^T * y2
$$

получаем, что при равных результатах у нас могут измениться веса. 

<a id='shag_3'></a>

## 3. Алгоритм преобразования

**Алгоритм**

1. Создадим матрицу-преобразователь.
2. Убедимся, что матрица-преобразователь соответствует условию обратимости.
3. Умножим исходную матрицу на матрица-преобразователь и получим датасет для обучения второй модели.
4. Обучим одну модель на матрицу до преобразования, а другую на матрице после преобразования.
5. Сравним эффективность модели по метрике r2_score.

Нам понадобится случайная квадратная обратимая матрица, которую мы будем умножать на исходный дата сет.

In [3]:
def matrix_creator(x):
    df = pd.DataFrame(np.random.randint(0,100,size=(x, x)))
    df_inv = pd.DataFrame(np.linalg.inv(df.values))
    print(df_inv*df)
    return df


**Обоснование**

В соответствии с приведенными в предыдущем пункте проекта результат работы модели на преобразованном дата сете не должен отличаться от предсказаний на исходном дата сете.

<a id='shag_4'></a>

## 4. Проверка алгоритма

In [4]:
f = df.drop(columns=['Страховые выплаты']).values 
t = df['Страховые выплаты'].values
print(f)

[[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 ...
 [0.00e+00 2.00e+01 3.39e+04 2.00e+00]
 [1.00e+00 2.20e+01 3.27e+04 3.00e+00]
 [1.00e+00 2.80e+01 4.06e+04 1.00e+00]]


In [5]:
preobrazovatel = matrix_creator(4)

          0         1         2         3
0 -0.302240  1.307286  1.912137 -3.087303
1  1.290898 -1.428291 -0.000000  0.855391
2  0.758918 -0.696872 -1.516977  2.595020
3 -0.319565  1.491737  0.922381 -0.562689


Матрица получилась обратимая

In [10]:
f2 = f @ preobrazovatel
t2 = t
print(f2)

              0          1          2          3
0     2184441.0   995050.0  4265695.0  3621930.0
1     1674273.0   763376.0  3268048.0  2775169.0
2      925392.0   422088.0  1806000.0  1533725.0
3     1835938.0   835640.0  3586296.0  3044663.0
4     1149752.0   524050.0  2244647.0  1906086.0
...         ...        ...        ...        ...
4995  1572274.0   716144.0  3070296.0  2606838.0
4996  2307297.0  1050512.0  4506448.0  3826069.0
4997  1492690.0   679568.0  2915496.0  2475238.0
4998  1440059.0   655810.0  2812391.0  2387793.0
4999  1787817.0   814114.0  3491695.0  2964605.0

[5000 rows x 4 columns]


In [12]:
model1 = LinearRegression()
model1.fit(f,t)
model2 = LinearRegression()
model2.fit(f2,t2)
test1 = model1.predict(df.drop(columns=['Страховые выплаты']).values)
test2 = model2.predict(df.drop(columns=['Страховые выплаты']).values @ preobrazovatel)
display(r2_score(df['Страховые выплаты'].values,test1))
display(r2_score(df['Страховые выплаты'].values,test2))

0.42494550286668

0.4249455028666934

После преобразования результат расчетов не изменился.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования